# Advent of Code 2022

## Day 13: Distress Signal

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

The hardest part of today's puzzle was the wording for the question:

*"If both values are lists, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input."*

It turns out you compare lengths only if the lists have all equal elements up until the point where one list runs out. This wording had me stuck for a long time. After resolving it, it was prety easy.

Part 1, I read the lines in pairs, and pass them to `eval`.

![After all... why not? Why shouldn't I eval?](https://i.redd.it/p3h0axhitl5a1.png)

(Above image found via u/Eclypse-Prime when browsing this subreddit afterwards)

I wrote a recursive cmp function which returns `-1`, `0`, or `1` even though these are out-of-fashion in Python 2. It turns out you don't need equality on the depth 0 call because all of the lists are unequal so you just need `<` but I did a full cmp function.

Part 2 was simply a matter of passing my cmp function to `cmp_to_key` and sorting.


### Imports

In [ ]:
from collections.abc import Iterator
from typing import Union, Optional
import functools

### Input File Reading

In [ ]:
# read lines in group of `group_size` lines at a time
def read_lines(filename: str,
               group_size: Optional[int] = None,
               discard_blank: bool = False) -> Iterator[Union[str, tuple[str]]]:
    assert (group_size is None) or (type(group_size) == int and group_size >= 2)
    with open(filename) as file:
        rv = []
        for line in file:
            line = line.strip()
            if line or not discard_blank:
                rv.append(line)
                if group_size is None or len(rv) == group_size:
                    if group_size:
                        yield tuple(rv)
                    else:
                        yield rv[0]
                    rv = []

        # make sure we don't leave any extra lines
        assert not rv, f'{len(rv)} left unprocessed at the end of the file!'

### The cmp function for comparing order of lists

In [ ]:
def compare(left: Union[list, int, str], right: Union[list, int, str]) -> int:

    # using eval to decode the strings
    if type(left) == str:
        left = eval(left)
    if type(right) == str:
        right = eval(right)

    # compare ints
    if type(left) == int and type(right) == int:
        if left < right:
            return -1
        if left > right:
            return 1
        return 0

    # wrap the int if not same type
    if type(left) == int and type(right) == list:
        return compare([left], right)
    if type(left) == list and type(right) == int:
        return compare(left, [right])

    # element-wise compare
    for l, r in zip(left, right):
        cmp = compare(l, r)
        if cmp:
            return cmp

    # If the left list runs out of items first, the inputs are in the right order.
    if len(right) > len(left):
        return -1

    # If the right list runs out of items first, the inputs are not in the right order.
    if len(right) < len(left):
        return 1

    # all elements match, same length
    return 0

### Part 1

In [ ]:
INPUT_FILE = 'data/input13.txt'

In [ ]:
def main():

    index = 1
    total = 0

    for left, right in read_lines(INPUT_FILE, group_size=2, discard_blank=True):
        if compare(left, right) <= 0:
            total += index
        index += 1

    print(f'The sum of the indices of pairs in the right order is {total}')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
DIVIDER_1 = [[2]]
DIVIDER_2 = [[6]]

In [ ]:
def main():

    lines = list(read_lines(INPUT_FILE, discard_blank=True))
    lines.append(DIVIDER_1)
    lines.append(DIVIDER_2)

    lines.sort(key=functools.cmp_to_key(compare))

    index1 = lines.index(DIVIDER_1) + 1
    index2 = lines.index(DIVIDER_2) + 1

    decoder_key = index1 * index2

    print(f'The decoder key for the distress signal is {decoder_key}')

In [ ]:
if __name__ == '__main__':
    main()